# **Bienvenidos a la clase sobre API REST**

### Objetivos de la Clase:

- Comprender qué es una API REST y cómo funciona en el contexto de la obtención de datos.
- Aprender a conectarse a una API usando Python (para lo cual volveremos a utilizar el metodo `requests`).
- Continuar familiarizandose con el formato JSON, que es la respuesta típica de muchas APIs.
- Hacer un ejercicio práctico usando la API de datos.gob.ar para normalizar la escritura de algunos domicilios y obtener sus coordenadas.

**¿Qué es REST(Representational State Transfer)?** Es un estilo de arquitectura que permite acceder y manipular recursos mediante URLs.

Vamos a trabajar con datos del gobierno de Argentina utilizando una API REST

https://datos.gob.ar/


Documentación:

https://datosgobar.github.io/georef-ar-api/

https://datosgobar.github.io/georef-ar-api/open-api/#/

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import requests
import pandas as pd


In [ ]:
# Función para solicitarle a la API el listado de provincias
def get_provincias():
    url = "https://apis.datos.gob.ar/georef/api/provincias"
    response = requests.get(url)
    return response.json()['provincias']


In [ ]:
get_provincias()

[{'centroide': {'lat': -34.6144420654301, 'lon': -58.4458763250916},
  'id': '02',
  'nombre': 'Ciudad Autónoma de Buenos Aires'},
 {'centroide': {'lat': -38.6419828626673, 'lon': -70.1198972237318},
  'id': '58',
  'nombre': 'Neuquén'},
 {'centroide': {'lat': -33.7611035381154, 'lon': -66.0252312714021},
  'id': '74',
  'nombre': 'San Luis'},
 {'centroide': {'lat': -30.7088227091528, 'lon': -60.9506872769706},
  'id': '82',
  'nombre': 'Santa Fe'},
 {'centroide': {'lat': -29.6849372775783, 'lon': -67.1817575814487},
  'id': '46',
  'nombre': 'La Rioja'},
 {'centroide': {'lat': -27.3359537960762, 'lon': -66.9478972451295},
  'id': '10',
  'nombre': 'Catamarca'},
 {'centroide': {'lat': -26.948283501723, 'lon': -65.3647655803683},
  'id': '90',
  'nombre': 'Tucumán'},
 {'centroide': {'lat': -26.3869871835867, 'lon': -60.765116260356},
  'id': '22',
  'nombre': 'Chaco'},
 {'centroide': {'lat': -24.8950871761481, 'lon': -59.9321901121647},
  'id': '34',
  'nombre': 'Formosa'},
 {'centroide

In [ ]:
# Función para solicitarle a la API n departamentos de una provincia en particular
def get_departamentos(provincia, n):
    url = f"https://apis.datos.gob.ar/georef/api/departamentos?provincia={provincia}&max={n}"
    response = requests.get(url)
    return response.json()['departamentos']




In [ ]:
get_departamentos("Buenos Aires", 3)

[{'centroide': {'lat': -37.9646159068483, 'lon': -60.2482821323384},
  'id': '06014',
  'nombre': 'Adolfo Gonzales Chaves',
  'provincia': {'id': '06', 'nombre': 'Buenos Aires'}},
 {'centroide': {'lat': -35.676806451923, 'lon': -59.7026607444531},
  'id': '06707',
  'nombre': 'Saladillo',
  'provincia': {'id': '06', 'nombre': 'Buenos Aires'}},
 {'centroide': {'lat': -36.015826657022, 'lon': -59.1764609497072},
  'id': '06455',
  'nombre': 'Las Flores',
  'provincia': {'id': '06', 'nombre': 'Buenos Aires'}}]

In [ ]:
# Función para solicitarle a la API n localidades de una provincia y departamento en particular

def get_localidades(provincia, departamento, n):
    url = f"https://apis.datos.gob.ar/georef/api/localidades?provincia={provincia}&departamento={departamento}&campos=nombre&max={n}"
    response = requests.get(url)
    return response.json()['localidades']


In [ ]:
provincia = "Buenos Aires"
departamento = "La Plata"
localidades = get_localidades(provincia, departamento, 4)
# Convertimos la lista de localidades a un DataFrame de pandas
df_localidades = pd.DataFrame(localidades)

# Agregar las columnas de provincia y departamento
df_localidades['provincia'] = provincia
df_localidades['departamento'] = departamento


df_localidades

,id,nombre,provincia,departamento
0,0644103006,Barrio Gambier,Buenos Aires,La Plata
1,0644103001,Abasto,Buenos Aires,La Plata
2,0644103002,Ángel Etcheverry,Buenos Aires,La Plata
3,0644103003,Arana,Buenos Aires,La Plata


In [ ]:
# La función get_direccion nos devuelve la direccion normalizada con su latittud y longitud

def get_direccion(provincia, direccion, departamento=None, localidad=None):
    if provincia == "Ciudad Autónoma de Buenos Aires":
        url = f"https://apis.datos.gob.ar/georef/api/direcciones?provincia=CABA&direccion={direccion}"
    elif departamento is not None:
        url = f"https://apis.datos.gob.ar/georef/api/direcciones?provincia={provincia}&departamento={departamento}&localidad={localidad}&direccion={direccion}"
    else:
        url = f"https://apis.datos.gob.ar/georef/api/direcciones?provincia={provincia}&localidad={localidad}&direccion={direccion}"
    response = requests.get(url)
    data = response.json()
    if 'direcciones' in data and data['direcciones']:
        return data['direcciones'][0], url
    return None, url


In [ ]:

# Función para

def details_as_table(data):

    df = pd.DataFrame([
        [
            f"{data['altura']['valor']} {data['altura']['unidad'] if data['altura']['unidad'] else ''}" if data['altura']['valor'] else "N/A",
            data['calle']['nombre'],
            data['departamento']['nombre'] if data['departamento']['nombre'] else "N/A",
            data['localidad_censal']['nombre'] if data['localidad_censal']['nombre'] else "N/A",
            f"{data['ubicacion']['lat']:.3f}, {data['ubicacion']['lon']:.3f}"
        ]], columns=['Altura', 'Calle', 'Departamento', 'Localidad', 'Coordenadas']
    ).T
    df.columns = [""]
    return df.T


In [ ]:
direccion = get_direccion("Buenos Aires", "Calle 6 777", "La Plata", "La Plata")


In [ ]:
direccion

({'altura': {'unidad': None, 'valor': 777},
  'calle': {'categoria': 'CALLE', 'id': '0644103003625', 'nombre': 'CALLE 6'},
  'calle_cruce_1': {'categoria': None, 'id': None, 'nombre': None},
  'calle_cruce_2': {'categoria': None, 'id': None, 'nombre': None},
  'departamento': {'id': '06441', 'nombre': 'La Plata'},
  'localidad_censal': {'id': '06441030', 'nombre': 'La Plata'},
  'nomenclatura': 'CALLE 6 777, La Plata, Buenos Aires',
  'piso': None,
  'provincia': {'id': '06', 'nombre': 'Buenos Aires'},
  'ubicacion': {'lat': -34.89362772186887, 'lon': -57.97587547108113}},
 'https://apis.datos.gob.ar/georef/api/direcciones?provincia=Buenos Aires&departamento=La Plata&localidad=La Plata&direccion=Calle 6 777')

In [ ]:
details_as_table(direccion[0])

,Altura,Calle,Departamento,Localidad,Coordenadas
,777,CALLE 6,La Plata,La Plata,"-34.894, -57.976"


In [ ]:
# Crear widgets
provincia_widget = widgets.Text(description="Provincia:")
localidad_widget = widgets.Text(description="Localidad:")
direccion_widget = widgets.Text(description="Direccion:")

button = widgets.Button(description="Obtener Coordenadas")

# Mostrar widgets
display(provincia_widget, localidad_widget, direccion_widget, button)

# Función de manejo del evento al presionar el botón
def on_button_click(b):
    provincia = provincia_widget.value
    localidad = localidad_widget.value
    direccion = direccion_widget.value
    data, url = get_direccion(provincia, direccion, localidad=localidad)

    if data:
        print(f"Coordenadas: {data}")
        #print(f"Coordenadas: {data['ubicacion']['lat']:.3f}, {data['ubicacion']['lon']:.3f}")
    else:
        print("No se encontraron coordenadas para la dirección proporcionada.")
    print(f"Consulta realizada con URL: {url}")

# Asignar la función al evento click del botón
button.on_click(on_button_click)

Text(value='', description='Provincia:')

Text(value='', description='Localidad:')

Text(value='', description='Direccion:')

Button(description='Obtener Coordenadas', style=ButtonStyle())

Coordenadas: -34.542, -58.489
Consulta realizada con URL: https://apis.datos.gob.ar/georef/api/direcciones?provincia=Buenos Aires&localidad=Vicente Lopez&direccion=sahores 2490
